In [ ]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

In [ ]:
cd '/content/drive/MyDrive/NLP/NLP project/legal_NER-main'

In [ ]:
!pip install -r requirements.txt

In [2]:
import pandas as pd

with open('/content/NER_TRAIN_JUDGEMENT.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('csvfile.csv', encoding='utf-8', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df

,id,annotations,data,meta
0,90d9a97c7b7749ec8a4f460fda6f937e,"[{'result': [{'value': {'start': 90, 'end': 10...",{'text': ' (7) On specific query by the Bench...,{'source': 'tax_districtcourts judgement https...
1,a325c57ba5b84c6fa46bee65e6616633,"[{'result': [{'value': {'start': 26, 'end': 30...",{'text': 'He was also asked whether Agya <span...,{'source': 'criminal_punjab-haryana_high_court...
2,33cface31108441080976035ee04d07e,"[{'result': [{'value': {'start': 13, 'end': 25...","{'text': ' 5.2 CW3 Mr Vijay Mishra , Deputy M...",{'source': 'financial_districtcourts judgement...
3,08debe900b51464e9f26b5d1eecae170,[{'result': []}],{'text': 'You are hereby asked not to carry ou...,{'source': 'civil_bombay_high_court judgement ...
4,428070222cd940eba78b233829eb28b8,"[{'result': [{'value': {'start': 18, 'end': 43...",{'text': 'The pillion rider T.V. Satyanarayana...,{'source': 'motorvehicles_andhra_high_court ju...
...,...,...,...,...
9430,5cb430a6d0e14e57a968f0c43d219682,"[{'result': [{'value': {'start': 202, 'end': 2...",{'text': 'It is prayed in the application that...,{'source': 'civil_districtcourts judgement htt...
9431,6e9147c9e6344865a8b095fb7f43f7c2,"[{'result': [{'value': {'start': 26, 'end': 35...","{'text': 'In the first instance, Mr.A.D.Desai ...",{'source': 'tax_gujarat_high_court judgement h...
9432,657ef5df817d4e95a36d702b4e41ce7d,"[{'result': [{'value': {'start': 0, 'end': 5, ...","{'text': 'Sikri, J. (as he then was), speaking...",{'source': 'constitution_allahabad_high_court ...
9433,9ff4b03ee09f49a6814b9550c145ea1e,"[{'result': [{'value': {'start': 1, 'end': 17,...",{'text': '(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C....,{'source': 'industrial&labour_madhya_pradesh_h...


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Text Preprocessing

### Preproceesing annotations start

In [5]:
annotations_column = df['annotations']

In [6]:
annotations_column =  annotations_column.to_numpy()

In [7]:
annotation_start = []
annotation_end = []


for sent in annotations_column:
  sent_start = []
  sent_end = []
  sent_text = []
  sent_label = []
  for sent_res in sent:
    for res_value in sent_res['result']:
      sent_start.append(res_value['value']['start'])

    for res_value in sent_res['result']:
      sent_end.append(res_value['value']['end'])
    


  annotation_start.append(sent_start)
  annotation_end.append(sent_end)


In [8]:
annotation_start[0]

[90, 267]

In [9]:
len(annotation_end)

9435

In [10]:
annotation_text = []
annotation_label = []
for sent in annotations_column:
    sent_text = []
    sent_label = []
    for sent_res in sent:

        for res_value in sent_res['result']:
          sent_text.append(res_value['value']['text'])
        
        for res_value in sent_res['result']:
          sent_label.append(res_value['value']['labels']) 

    annotation_text.append(sent_text)
    annotation_label.append(sent_label)

In [11]:
annotation_text[0]

['Hongkong Bank', 'Rahul & Co.']

In [12]:
annotation_label[0]

[['ORG'], ['ORG']]

In [13]:
column_names = ['start', 'end' , 'text' , 'labels']

final_df = pd.DataFrame()
final_df['start'] = annotation_start
final_df['end'] = annotation_end
final_df['text'] = annotation_text
final_df['labels'] = annotation_label

final_df.head()

,start,end,text,labels
0,"[90, 267]","[103, 278]","[Hongkong Bank, Rahul & Co.]","[[ORG], [ORG]]"
1,"[26, 101, 159]","[30, 105, 174]","[Agya, Kaur, Tarlochan Singh]","[[OTHER_PERSON], [OTHER_PERSON], [OTHER_PERSON]]"
2,"[13, 44, 62, 121]","[25, 60, 64, 130]","[Vijay Mishra, HDFC Bank, Noida, UP, HDFC Bank]","[[WITNESS], [ORG], [GPE], [ORG]]"
3,[],[],[],[]
4,[18],[43],[T.V. Satyanarayana Murthy],[[OTHER_PERSON]]


In [14]:
for a,b,c,d in zip(annotation_start[:2], annotation_end[:2],annotation_text[:2],annotation_label[:2]):
  print(a,b,c,d)

[90, 267] [103, 278] ['Hongkong Bank', 'Rahul & Co.'] [['ORG'], ['ORG']]
[26, 101, 159] [30, 105, 174] ['Agya', 'Kaur', 'Tarlochan Singh'] [['OTHER_PERSON'], ['OTHER_PERSON'], ['OTHER_PERSON']]


In [15]:
zip(annotation_start[0], annotation_end[0])

In [16]:
pwd

'/content'

In [17]:
final_df.to_csv('Processed_DataFrame_Train')

## IOB tagging

In [18]:
import pandas as pd

df_ = pd.read_csv('/content/Processed_DataFrame_Train')
df_.head()

,Unnamed: 0,start,end,text,labels
0,0,"[90, 267]","[103, 278]","['Hongkong Bank', 'Rahul & Co.']","[['ORG'], ['ORG']]"
1,1,"[26, 101, 159]","[30, 105, 174]","['Agya', 'Kaur', 'Tarlochan Singh']","[['OTHER_PERSON'], ['OTHER_PERSON'], ['OTHER_P..."
2,2,"[13, 44, 62, 121]","[25, 60, 64, 130]","['Vijay Mishra', 'HDFC Bank, Noida', 'UP', 'HD...","[['WITNESS'], ['ORG'], ['GPE'], ['ORG']]"
3,3,[],[],[],[]
4,4,[18],[43],['T.V. Satyanarayana Murthy'],[['OTHER_PERSON']]


In [19]:
text = df.iloc[[0]]['data']

In [20]:
text[0]['text']

"\n\n(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy."

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(text[0]['text'])

for d in doc:
  print(d.text)




(
7
)
On
specific
query
by
the
Bench
about
an
entry
of
Rs
.
1,31,37,500
on
deposit
side
of
Hongkong
Bank
account
of
which
a
photo
copy
is
appearing
at
p.
40
of
assessee
's
paper
book
,
learned
authorised
representative
submitted
that
it
was
related
to
loan
from
broker
,
Rahul
&
Co.
on
the
basis
of
his
submission
a
necessary
mark
is
put
by
us
on
that
photo
copy
.


In [22]:
import re
t = re.sub('\n', " ", text[0]['text'])


In [23]:
" ".join(t.split())

"(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy."

In [24]:
df['data']

0       {'text': '

(7) On specific query by the Bench...
1       {'text': 'He was also asked whether Agya <span...
2       {'text': ' 
5.2 CW3 Mr Vijay Mishra , Deputy M...
3       {'text': 'You are hereby asked not to carry ou...
4       {'text': 'The pillion rider T.V. Satyanarayana...
                              ...                        
9430    {'text': 'It is prayed in the application that...
9431    {'text': 'In the first instance, Mr.A.D.Desai ...
9432    {'text': 'Sikri, J. (as he then was), speaking...
9433    {'text': '(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C....
9434    {'text': 'In the cross-examination, PW.12 has ...
Name: data, Length: 9435, dtype: object

In [25]:
def preprocess(sent):
  sent = re.sub('\n', " ", sent)
  sent = sent.replace("...", " ")
  sent = " ".join(sent.split())

  return sent


clean_text = []
for i in df['data']:
  clean_text.append(preprocess(i['text']))

In [26]:
len(clean_text)

9435

In [27]:
df_['clean_text'] = clean_text

In [28]:
df_.head()

,Unnamed: 0,start,end,text,labels,clean_text
0,0,"[90, 267]","[103, 278]","['Hongkong Bank', 'Rahul & Co.']","[['ORG'], ['ORG']]",(7) On specific query by the Bench about an en...
1,1,"[26, 101, 159]","[30, 105, 174]","['Agya', 'Kaur', 'Tarlochan Singh']","[['OTHER_PERSON'], ['OTHER_PERSON'], ['OTHER_P...","He was also asked whether Agya <span class=""hi..."
2,2,"[13, 44, 62, 121]","[25, 60, 64, 130]","['Vijay Mishra', 'HDFC Bank, Noida', 'UP', 'HD...","[['WITNESS'], ['ORG'], ['GPE'], ['ORG']]","5.2 CW3 Mr Vijay Mishra , Deputy Manager, HDFC..."
3,3,[],[],[],[],You are hereby asked not to carry out any cons...
4,4,[18],[43],['T.V. Satyanarayana Murthy'],[['OTHER_PERSON']],The pillion rider T.V. Satyanarayana Murthy al...


In [29]:
s = df_.iloc[0]['clean_text']

for i in range(7,40):
  print(s[i])

s
p
e
c
i
f
i
c
 
q
u
e
r
y
 
b
y
 
t
h
e
 
B
e
n
c
h
 
a
b
o
u
t


In [30]:
df_.iloc[1]['clean_text']

import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(df_.iloc[0]['clean_text'])

print(doc[0])

(


In [31]:
span = doc.char_span(73,100,label = "COURT")
span

## New solution

In [32]:
def sentence_cleaner(sentence):
    '''
    replacing the unwanted charachters
    '''
    reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
    return " ".join(reg.findall(sentence.replace('\n', ' '))).replace(" / ", '/')

In [33]:
def sentence_cleaner_v1(sentence):
    '''
    replacing the unwanted charachters
    '''
    sent_1 = " ".join(sentence.replace("\n" , " ").replace("..." , " ").replace("-", " ").replace("," , " ").replace(":" , " ").replace("=",  " ").split())
    doc = nlp(sent_1)
    doc_list = []
    for i in doc:
      doc_list.append(str(i))
    return doc_list

In [34]:
def sentence_tagging(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,temp_len)
    if temp_len > 1 :
        for i in range(temp_len):
            #print(i, temp_len - 1)
            if i == 0 :
              word2ent[temp_sent[i]] =  "B-" + ann['value']['labels'][0]
            
            elif i == (temp_len - 1):
              word2ent[temp_sent[i]] =  "E-" + ann['value']['labels'][0]

            else :
              word2ent[temp_sent[i]] =  "I-" + ann['value']['labels'][0]

    else :
        word2ent[temp_sent[0]] = "B-" + ann['value']['labels'][0]

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys() and word + ',' not in word2ent.keys() :
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
        tag_list.append(word2ent[word])
      
      else :
          tag_list.append(word2ent[word])
  
  
  print(word2ent )
  return tag_list
      

In [35]:
def sentence_tagging_v1(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
      tag_list.append(word2ent[word])
      
  
  
  #print(word2ent )
  return tag_list
      

In [36]:
def sentence_tagging_v2(sent, annotation):

  """
  current using word
  """

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = sentence_cleaner_v1(sent[ ann['value']['start']: ann['value']['end']])
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner_v1(sent)
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        tag = "O"
      else :
        if len(word2ent[word]) != 0 :
          tag = word2ent[word].pop(0)
        else :
          tag = "O"

      #print(word,tag)   
      tag_list.append(str(tag))
      
  
  
  #print(word2ent )
  return tag_list
      

In [37]:
IOBE_tagging = []
i = 0
for sent, anno in zip( df['data'].to_list(), df['annotations'].to_list()):
    IOBE_tagging.append(sentence_tagging_v2(sent,anno))
    
  

In [38]:
IOBE_tagging[2]

['O',
 'O',
 'O',
 'B-WITNESS',
 'E-WITNESS',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'E-ORG',
 'B-GPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'E-ORG',
 'O',
 'O',
 'O',
 'O',
 'O']

In [39]:
final_df['tags'] = IOBE_tagging
final_df.head()

,start,end,text,labels,tags
0,"[90, 267]","[103, 278]","[Hongkong Bank, Rahul & Co.]","[[ORG], [ORG]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[26, 101, 159]","[30, 105, 174]","[Agya, Kaur, Tarlochan Singh]","[[OTHER_PERSON], [OTHER_PERSON], [OTHER_PERSON]]","[O, O, O, O, O, B-OTHER_PERSON, O, O, O, O, O,..."
2,"[13, 44, 62, 121]","[25, 60, 64, 130]","[Vijay Mishra, HDFC Bank, Noida, UP, HDFC Bank]","[[WITNESS], [ORG], [GPE], [ORG]]","[O, O, O, B-WITNESS, E-WITNESS, O, O, B-ORG, I..."
3,[],[],[],[],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,[18],[43],[T.V. Satyanarayana Murthy],[[OTHER_PERSON]],"[O, O, O, B-OTHER_PERSON, I-OTHER_PERSON, E-OT..."


In [40]:
sent_1 = df['data'].to_list()[2]['text']
sent_1

' \n5.2 CW3 Mr Vijay Mishra , Deputy Manager, HDFC Bank, Noida, UP has deposed that complainant had a current account with HDFC Bank in the year 2004\xad2005.'

In [ ]:
# tag_2 = final_df['tags'].to_list()[77]
# tag_2

In [41]:
# [\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]

reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
" ".join(reg.findall(sent_1.replace('\n', ' '))).replace(" / ", '/')

'5 2 CW3 Mr Vijay Mishra Deputy Manager HDFC Bank Noida UP has deposed that complainant had a current account with HDFC Bank in the year 2004 2005'

In [42]:
" ".join(sent_1.replace(" "," ").split())

'5.2 CW3 Mr Vijay Mishra , Deputy Manager, HDFC Bank, Noida, UP has deposed that complainant had a current account with HDFC Bank in the year 2004\xad2005.'

In [43]:
word2count = {}

for sent in df['data'].to_list():
    for word in sent['text'].split():
        if word not in word2count:
          word2count[word] = 0
        else:
          word2count[word] += 1

In [44]:
for key in word2count.keys():
  if re.search(r'\bM+\b', key):
    print(key)

A.M.
S.M.
MM­07,
(M.Cr.C.
M.
M/S
M.B.
(M.
M.V.C.No.6640/2013:
CM(M)527/1999.
V.M.
(M.P.)
M/s.
M.Y.
PW25-Dr.M.Sathikumar,
M.Cr.C.
MM
M.P.S.R.T.C.
C.M.M.,
M/s
M.M.
P.M.
M
1996#1996#M/s
M.P.L.J.
M.A.C.A
M.V.
M.P.
M.V
M/s.Nandh
Crl.M.A.
N.M.
CW1/M
M.L.C.
(M),
M.Sc.
M.A.
M.C.
M/
M.A.C.
(M-4
CM(M)
(O&M)
M.O.
Mr.I.M.Nanavati
A.D.M.
P.M.,
C.M.A.
K.M.
M.O.59,
M.O.60,
MHC(M)
MHC(M).
(M.L.
M/s.Jayanthilal
M.K.
M.V.Act,
M/s.Amigo
M.Os.
M.S.RAMACHANDRA
Crl.M.P.No.3726
M/s.Tirumalas
(M.P.
Cr.M.C.
M.C.O.P.No.1326
(M)}.
Crl.M.C.1161/2007
M.A
(C.M.A
B.M.T.
H.M.T.
M.Venugopal
M/s.Siva
(M.O.2),
(M.O.5),
(M.O.1),
(M.O.3)
(M.O.4)
(M/s.
(M/B)
C.M.No.15703
M.R.,
M.V.Act
O.M.No.
No.22148-M
Crl.M.C
J.M.F.C.,
M.L.Mahajan
Crl.M.A.No.1888
C.M.P.
C.J.M.,
(P.M.Kuriakose
M.C.O.P.
M.V.C.No.6642/2013
Crl.M.C.
M.Parasakthi
M/s.Praveen
M/s.Thirumala's.
M.S.
Crl.M.P.No.569
R.M.
M.A.C.A.No.850/2016
M.P.Municipal
M.A.C.T
C.M.V.
D.C.M.
T.M.A.
M.S.,
M.D.
(M/s
M/30yrs
M'Naughton'
Crl.M.A.7959/2011
M.C.C.No.500/2015
P.M
M.P.,


## Using Tokenizer

In [45]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [46]:
sent_1 = " ".join(sent_1.replace("\n" , " ").replace("..." , " ").replace("-", "").split())
doc = nlp(sent_1)
doc_list = []
for i in doc:
  doc_list.append(str(i))
doc_list

['5.2',
 'CW3',
 'Mr',
 'Vijay',
 'Mishra',
 ',',
 'Deputy',
 'Manager',
 ',',
 'HDFC',
 'Bank',
 ',',
 'Noida',
 ',',
 'UP',
 'has',
 'deposed',
 'that',
 'complainant',
 'had',
 'a',
 'current',
 'account',
 'with',
 'HDFC',
 'Bank',
 'in',
 'the',
 'year',
 '2004\xad2005',
 '.']

# Clean the sentences

In [47]:
clean_sentences = []

for sent in df['data'].to_list():
    clean_sentences.append(sentence_cleaner_v1(sent['text']))



In [48]:
final_df['clean_sent'] = clean_sentences
final_df.head()

,start,end,text,labels,tags,clean_sent
0,"[90, 267]","[103, 278]","[Hongkong Bank, Rahul & Co.]","[[ORG], [ORG]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[(, 7, ), On, specific, query, by, the, Bench,..."
1,"[26, 101, 159]","[30, 105, 174]","[Agya, Kaur, Tarlochan Singh]","[[OTHER_PERSON], [OTHER_PERSON], [OTHER_PERSON]]","[O, O, O, O, O, B-OTHER_PERSON, O, O, O, O, O,...","[He, was, also, asked, whether, Agya, <, span,..."
2,"[13, 44, 62, 121]","[25, 60, 64, 130]","[Vijay Mishra, HDFC Bank, Noida, UP, HDFC Bank]","[[WITNESS], [ORG], [GPE], [ORG]]","[O, O, O, B-WITNESS, E-WITNESS, O, O, B-ORG, I...","[5.2, CW3, Mr, Vijay, Mishra, Deputy, Manager,..."
3,[],[],[],[],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[You, are, hereby, asked, not, to, carry, out,..."
4,[18],[43],[T.V. Satyanarayana Murthy],[[OTHER_PERSON]],"[O, O, O, B-OTHER_PERSON, I-OTHER_PERSON, E-OT...","[The, pillion, rider, T.V., Satyanarayana, Mur..."


## Word2idx

In [49]:
def get_dict_map(df , token_or_tag):
    """
    converts word to index

    [Need changes !!!]
    """
    tok2idx = {}
    idx2tok = {}

    vocab = []

    if token_or_tag == 'token':
        for sent in df['clean_sent'].to_list():
          vocab.extend( list(set(sent)) )

        vocab = set(vocab)

    else :
        for sent in df['tags'].to_list():
          vocab.extend( list(set(sent)) )
        
        vocab = set(vocab)


    idx2tok = {idx:tok for idx , tok in enumerate(vocab)}
    tok2idx = {tok:idx for idx , tok in enumerate(vocab)}

    return tok2idx, idx2tok



word2idx, idx2word = get_dict_map(final_df, 'token')
tag2idx, idx2tag = get_dict_map(final_df, 'tag')


In [50]:
"""
Padding and converting from word sequence to index sequence

"""

max_len = max([len(sent) for sent in final_df['clean_sent'].tolist()])
pad_idx = len(word2idx)
tag_pad_idx = tag2idx['O']
word_idx_sequence = []
tag_idx_sequence = []
pad = "PAD"

for sent in final_df['clean_sent'].to_list():
    #print(sent)
    word_idx_sequence.append([word2idx[word] for word in sent] + [pad_idx for _ in range(max_len - len(sent))])


for tag_seq in final_df['tags'].to_list():
    tag_idx_sequence.append([tag2idx[tag] for tag in tag_seq] + [tag_pad_idx for _ in range(max_len - len(tag_seq))])


In [51]:
final_df['word_idx'] = word_idx_sequence
final_df['tag_idx'] = tag_idx_sequence



In [52]:
len(tag2idx)

40

In [53]:
tag2idx

{'E-RESPONDENT': 0,
 'B-RESPONDENT': 1,
 'O': 2,
 'B-GPE': 3,
 'I-OTHER_PERSON': 4,
 'B-PROVISION': 5,
 'E-ORG': 6,
 'E-PROVISION': 7,
 'E-GPE': 8,
 'B-STATUTE': 9,
 'B-JUDGE': 10,
 'B-ORG': 11,
 'E-JUDGE': 12,
 'B-OTHER_PERSON': 13,
 'I-WITNESS': 14,
 'I-DATE': 15,
 'I-PROVISION': 16,
 'I-COURT': 17,
 'I-CASE_NUMBER': 18,
 'B-PETITIONER': 19,
 'I-PRECEDENT': 20,
 'I-JUDGE': 21,
 'E-CASE_NUMBER': 22,
 'I-ORG': 23,
 'E-OTHER_PERSON': 24,
 'B-WITNESS': 25,
 'I-PETITIONER': 26,
 'E-WITNESS': 27,
 'E-PRECEDENT': 28,
 'B-CASE_NUMBER': 29,
 'B-PRECEDENT': 30,
 'I-GPE': 31,
 'I-RESPONDENT': 32,
 'B-DATE': 33,
 'E-DATE': 34,
 'I-STATUTE': 35,
 'B-COURT': 36,
 'E-COURT': 37,
 'E-PETITIONER': 38,
 'E-STATUTE': 39}

In [54]:
import json

with open("tag2idx_Judgement","w") as file:
  json.dump(tag2idx, file)

In [55]:
import json

with open("idx2tag_Judgement","w") as file:
  json.dump(idx2tag, file)

In [56]:
word2idx

{'unrewarding': 0,
 'overpayment': 1,
 'purses': 2,
 'No.216': 3,
 '1635': 4,
 '1775': 5,
 'Sanjay': 6,
 'mandates': 7,
 'deliberation': 8,
 'Commenting': 9,
 'MCrC': 10,
 'Ramachandran': 11,
 'Lakhawat': 12,
 '1619': 13,
 'PIL': 14,
 'apportioned': 15,
 'galloping': 16,
 'Nos.19': 17,
 '2088/9': 18,
 'Thomas': 19,
 'Sukhdev': 20,
 'acquiring': 21,
 'disallow': 22,
 'stall': 23,
 '26.11.2005': 24,
 'Satyanand': 25,
 'pellant': 26,
 'Mookerjee': 27,
 'Arijit': 28,
 'best': 29,
 'forgery': 30,
 'Deface': 31,
 'pigeon': 32,
 'docs': 33,
 'University': 34,
 'vend': 35,
 'sore': 36,
 'through': 37,
 'Payment': 38,
 'destination': 39,
 'illegally': 40,
 '18.10.2008': 41,
 'Bind': 42,
 'Energy': 43,
 'achieve': 44,
 'anxiety': 45,
 'guns': 46,
 'rain': 47,
 'Simranjit': 48,
 'C.T.Scan': 49,
 'Marine': 50,
 'determines': 51,
 'Haji': 52,
 '74/08': 53,
 'No.10017': 54,
 'Ghosh': 55,
 'deposition': 56,
 '574': 57,
 'Satyanarayana': 58,
 'possess': 59,
 'UTTAR': 60,
 'statistical': 61,
 'Hanuman'

In [57]:
final_df.to_csv("Final_DataFrame_Train_Judgement")